In [30]:
# from transformers.data.processors.squad import SquadResult, SquadV1Processor

# processor = SquadV1Processor()
# examples = processor.get_dev_examples(args.data_dir, filename=args.predict_file)

In [31]:
import torch
import json
import collections
from squad import SquadResult, SquadV1Processor, SquadV2Processor
from squad_metrics import (
    compute_predictions_log_probs,
    compute_predictions_logits,
    squad_evaluate,
)

cached_features_file='tydiqa/cached_dev_tydiqa-gold_384_'
pred_file='tydiqa/predictions_.json'
features_and_dataset = torch.load(cached_features_file)

examples = features_and_dataset["examples"]
predictions=json.load(open(pred_file))

In [ ]:
langs = ['swahili','bengali','arabic','korean','english','indonesian','japanese','russian',
        'telugu','finnish','thai']

for lan in langs:
    name = lan+'--'
    new_examples=[]
    new_preds=collections.OrderedDict()
    for exam in examples:
        if exam.qas_id.startswith(name)==True:
            new_examples.append(exam)
            new_preds[exam.qas_id]=predictions[exam.qas_id]
    if len(new_examples)!=0:
        results = squad_evaluate(new_examples, new_preds)
        print(name)
        print(results)

for lan in langs[:-1]:
    name = lan+'-english--'
    new_examples=[]
    new_preds=collections.OrderedDict()
    for exam in examples:
        if exam.qas_id.startswith(name)==True:
            new_examples.append(exam)
            new_preds[exam.qas_id]=predictions[exam.qas_id]
    if len(new_examples)!=0:
        results = squad_evaluate(new_examples, new_preds)
        print(name)
        print(results)

for lan in langs[:-1]:
    name = 'english-'+lan+'--'
    new_examples=[]
    new_preds=collections.OrderedDict()
    for exam in examples:
        if exam.qas_id.startswith(name)==True:
            new_examples.append(exam)
            new_preds[exam.qas_id]=predictions[exam.qas_id]
    if len(new_examples)!=0:
        results = squad_evaluate(new_examples, new_preds)
        print(name)
        print(results)

In [3]:
import os 
import json
import pickle

def get_qdata(filename):

    with open(filename,'r') as f:
        data=json.loads(f.read())
    mlqa_q={}
    i=0
    for dat in data['data']:
    #     if i>2:
    #         break
    #     print(dat)
        for par in dat['paragraphs']:
            for qos in par['qas']:
                mlqa_q[qos['id']] = qos['question'].strip()
                i+=1
    print(i, len(mlqa_q))
    if i!=len(mlqa_q):
        mlqa_q={}
        i=0
        for dat in data['data']:
        #     if i>2:
        #         break
        #     print(dat)
            for par in dat['paragraphs']:
                for qos in par['qas']:
                    if qos['id'] not in mlqa_q:
                        mlqa_q[qos['id']]=[qos['question'].strip()]
                    else:
                        mlqa_q[qos['id']].append(qos['question'].strip())
                    i+=1
        print(i, len(mlqa_q))        
    return mlqa_q

In [11]:
from wikidata.client import Client
import json
import os

json_name = 'tydiqa_data/all_d.json'
with open(json_name) as inp:
    jsonstr = inp.read()
ent_ids = json.loads(jsonstr)

json_name = 'country_dict.json'
with open(json_name) as inp:
    jsonstr = inp.read()
countries = json.loads(jsonstr)

json_name = 'id2geo.json'
with open(json_name) as inp:
    jsonstr = inp.read()
id2geo = json.loads(jsonstr)

json_name = 'place2country.json'
with open(json_name) as inp:
    jsonstr = inp.read()
place2coun = json.loads(jsonstr)

def get_orig_id_ent(id_ques,all_q_id, all_ent):
    client = Client()
    orig_id_ent={}
    tags=['country','citizen','born','died']
    for ids,q in id_ques.items():
        lang=ids.split('-')[0]
        if q in all_q_id[lang]:
            ent=all_ent[lang][int(all_q_id[lang][q])]
            orig_id_ent[ids]=[]
            if ent in ent_ids:
    #             print(q,ent_ids[ent],ent)
                for x,j in ent_ids[ent].items():
                    if x in tags:
                        if j in countries: 
                            orig_id_ent[ids].append(countries[j])
                        elif j in id2geo:
                            orig_id_ent[ids].append(id2geo[j]['country'])
                        elif j in place2coun:
                            orig_id_ent[ids].append(place2coun[j]['country'])
                        else:
                            place = client.get('Q745956', load=True)
                            for key2 in place.keys():
                                if key2.id == 'P17':
                                    try:
                                        countryid = place[key2].id
                                        orig_id_ent[ids].append(countries[countryid])
#                                         print(x,j,countries[countryid])
                                    except:
                                        print(key2,  place)
                orig_id_ent[ids]=set(orig_id_ent[ids])
    return orig_id_ent

def get_coun2ids(orig_id_ent):
    coun2ids={}
    for ids, coun in orig_id_ent.items():
        if len(coun)!=0:
            for x in coun:
                if x not in coun2ids:
                    coun2ids[x]=[ids]
                else:
                    coun2ids[x].append(ids)
    return coun2ids

#### train file

In [27]:
import pickle

datapath='../data/sentences/tydiqa'


id_ques=get_qdata(os.path.join('tydiqa','tydiqa-goldp-v1.1-train.json'))
ques_id = {j:i for i,j in id_ques.items()}

all_q={}
all_q_id={}
all_q_list=[]
for f in os.listdir(datapath):
    if 'train' in f and 'train-bn' not in f:
        with open(os.path.join(datapath,f),'rb') as f1:
            sents=pickle.load(f1)
            sents=[x.replace('[START] ','').replace(' [END]','') for x in sents]
            name=f.split('.')[0].replace('tydiqa-train-','')
            all_q[name]=sents
            all_q_id[name]={x:i for i,x in enumerate(sents)}
            

import pickle

datapath='../data/entities/tydiqa'

all_ent={}
all_ent_id={}

for f in os.listdir(datapath):
    if 'train' in f and 'train-bn' not in f:
        with open(os.path.join(datapath,f),'rb') as f1:
            sents=pickle.load(f1)
            name=f.split('.')[0].replace('tydiqa-train-','')
            all_ent[name]=[x[0]['id'] if len(x)>=1 else '0' for x in sents]

49881 49881


In [28]:
orig_train = get_orig_id_ent(id_ques,all_q_id, all_ent)
coun2ids_train = get_coun2ids(orig_train)
langs = ['swahili','bengali','arabic','korean','english','indonesian','japanese','russian',
        'telugu','finnish','thai']
count_train={}
for lang in langs:
    count_train[lang]={}
    for countr,ids in coun2ids_train.items():
        for id_one in ids:
            if lang in id_one:
                if countr not in count_train[lang]:
                    count_train[lang][countr]=1
                else:
                    count_train[lang][countr]+=1

#### dev file

In [23]:
import pickle

datapath='../data/sentences/tydiqa'

types='dev'
id_ques=get_qdata(os.path.join('tydiqa','tydiqa-goldp-v1.1-{}.json'.format(types)))
ques_id = {j:i for i,j in id_ques.items()}

all_q={}
all_q_id={}
all_q_list=[]
for f in os.listdir(datapath):
    if types in f and 'train-bn' not in f:
        with open(os.path.join(datapath,f),'rb') as f1:
            sents=pickle.load(f1)
            sents=[x.replace('[START] ','').replace(' [END]','') for x in sents]
            name=f.split('.')[0].replace('tydiqa-{}-'.format(types),'')
            all_q[name]=sents
            all_q_id[name]={x:i for i,x in enumerate(sents)}
            

import pickle

datapath='../data/entities/tydiqa'

all_ent={}
all_ent_id={}

for f in os.listdir(datapath):
    if types in f and 'train-bn' not in f:
        with open(os.path.join(datapath,f),'rb') as f1:
            sents=pickle.load(f1)
            name=f.split('.')[0].replace('tydiqa-{}-'.format(types),'')
            all_ent[name]=[x[0]['id'] if len(x)>=1 else '0' for x in sents]

5077 5077


In [25]:
orig_dev = get_orig_id_ent(id_ques,all_q_id, all_ent)
coun2ids_dev = get_coun2ids(orig_dev)
langs = ['swahili','bengali','arabic','korean','english','indonesian','japanese','russian',
        'telugu','finnish','thai']
count_dev={}
for lang in langs:
    count_dev[lang]={}
    for countr,ids in coun2ids_dev.items():
        for id_one in ids:
            if lang in id_one:
                if countr not in count_dev[lang]:
                    count_dev[lang][countr]=1
                else:
                    count_dev[lang][countr]+=1

In [48]:
langs = ['swahili','bengali','arabic','korean','english','indonesian','japanese','russian',
        'telugu','finnish','thai']

dev_result={}
for lan in langs:
    name = lan+'-'
    dev_result[lan]={}
    for cname,ids in coun2ids_dev.items():
        new_examples=[]
        new_preds=collections.OrderedDict()
        for exam in examples:
            if exam.qas_id.startswith(name)==True and exam.qas_id in ids:
                new_examples.append(exam)
                new_preds[exam.qas_id]=predictions[exam.qas_id]
            
            
        if len(new_examples)!=0:
            results = squad_evaluate(new_examples, new_preds)
#             print(name,cname,len(new_examples))
#             print(results['f1'])
            dev_result[lan][cname]=results['f1']

In [50]:
# count_train,count_dev,dev_result

#### analysis

In [101]:
def get_within_between(orig_split, count_split,result,split):
    lang_trainc={}
    langs = ['swahili','bengali','arabic','korean','english','indonesian','japanese','russian',
            'telugu','finnish','thai']
    lang_trainc['total']=0
    for lang in langs:
        for x in orig_split.keys():
            if lang in x:
                if lang not in lang_trainc:
                    lang_trainc[lang]=1
                else:
                    lang_trainc[lang]+=1
        if lang in lang_trainc:
            lang_trainc['total']+=lang_trainc[lang]

    gby_coun={}
    for lang,coun in count_split.items():
        for coun,count in coun.items():
            if coun not in gby_coun:
                gby_coun[coun]={}
            gby_coun[coun][lang]={
                'count':count,
                'within_lang':round(100*(count/lang_trainc[lang]),2),
                'between_lang':round(100*(count/lang_trainc['total']),2)
            }
            if split=='dev':
                gby_coun[coun][lang]['f1']=result[lang][coun]
    return gby_coun

In [102]:
gby_countrain=get_within_between(orig_train, count_train,dev_result,'train')
gby_coundev=get_within_between(orig_dev, count_dev,dev_result,'dev')

In [107]:
import pandas as pd
pd.DataFrame.from_dict(gby_coundev).T

,swahili,arabic,korean,english,indonesian,russian,finnish,telugu,bengali
Italy,"{'count': 6, 'within_lang': 1.2, 'between_lang...","{'count': 18, 'within_lang': 1.95, 'between_la...","{'count': 5, 'within_lang': 1.81, 'between_lan...","{'count': 3, 'within_lang': 0.68, 'between_lan...","{'count': 7, 'within_lang': 1.24, 'between_lan...","{'count': 8, 'within_lang': 0.99, 'between_lan...","{'count': 12, 'within_lang': 1.53, 'between_la...",NaN,NaN
United States of America,"{'count': 26, 'within_lang': 5.21, 'between_la...","{'count': 30, 'within_lang': 3.26, 'between_la...","{'count': 30, 'within_lang': 10.87, 'between_l...","{'count': 47, 'within_lang': 10.68, 'between_l...","{'count': 26, 'within_lang': 4.6, 'between_lan...","{'count': 35, 'within_lang': 4.31, 'between_la...","{'count': 79, 'within_lang': 10.1, 'between_la...","{'count': 13, 'within_lang': 1.94, 'between_la...",NaN
Germany,"{'count': 9, 'within_lang': 1.8, 'between_lang...","{'count': 12, 'within_lang': 1.3, 'between_lan...","{'count': 8, 'within_lang': 2.9, 'between_lang...","{'count': 6, 'within_lang': 1.36, 'between_lan...","{'count': 14, 'within_lang': 2.48, 'between_la...","{'count': 21, 'within_lang': 2.59, 'between_la...","{'count': 66, 'within_lang': 8.44, 'between_la...","{'count': 1, 'within_lang': 0.15, 'between_lan...","{'count': 1, 'within_lang': 0.88, 'between_lan..."
Egypt,"{'count': 4, 'within_lang': 0.8, 'between_lang...","{'count': 16, 'within_lang': 1.74, 'between_la...",NaN,NaN,NaN,"{'count': 6, 'within_lang': 0.74, 'between_lan...","{'count': 2, 'within_lang': 0.26, 'between_lan...",NaN,NaN
Israel,"{'count': 11, 'within_lang': 2.2, 'between_lan...","{'count': 105, 'within_lang': 11.4, 'between_l...",NaN,NaN,"{'count': 4, 'within_lang': 0.71, 'between_lan...","{'count': 2, 'within_lang': 0.25, 'between_lan...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
German Democratic Republic,NaN,NaN,NaN,NaN,NaN,NaN,"{'count': 1, 'within_lang': 0.13, 'between_lan...",NaN,NaN
Classical Athens,NaN,NaN,NaN,NaN,NaN,NaN,"{'count': 1, 'within_lang': 0.13, 'between_lan...",NaN,NaN
São Tomé and Príncipe,NaN,NaN,NaN,NaN,NaN,NaN,"{'count': 1, 'within_lang': 0.13, 'between_lan...",NaN,NaN
Danish Realm,NaN,NaN,NaN,NaN,NaN,NaN,"{'count': 2, 'within_lang': 0.26, 'between_lan...",NaN,NaN
